# PINN Observer Demo Notebook

This notebook demonstrates the usage of the PINN Observer package for training and evaluating physics-informed neural network observers.

In [ ]:
# Import required packages
import numpy as np
import matplotlib.pyplot as plt
import torch

# Import PINN observer components
from pinn_observer import (
    generate_dataset, 
    PINNObserver, 
    train, 
    plot_pinn_observer_separate_figures,
    SYSTEM_MAP
)

print("Available systems:", list(SYSTEM_MAP.keys()))

## 1. Data Generation

Let's start by generating some training data for the modified academic system.

In [ ]:
# Generate dataset
system_name = "modified_academic"
t_colloc, x_colloc, y_colloc, t_bc, x_bc0 = generate_dataset(
    name=system_name,
    t0=0.0,
    t_end=10.0,
    N_colloc=1000,
    N_bc=10,
    seed=42
)

print(f"Generated data for {system_name}:")
print(f"  Collocation points: {len(t_colloc)}")
print(f"  State dimension: {x_colloc.shape[1]}")
print(f"  Measurement dimension: {y_colloc.shape[1]}")
print(f"  Time range: [{t_colloc.min():.2f}, {t_colloc.max():.2f}]")

In [ ]:
# Visualize the true trajectories
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

for i in range(x_colloc.shape[1]):
    axes[i].plot(t_colloc, x_colloc[:, i], 'b-', label=f'$x_{i+1}$(t)')
    axes[i].set_ylabel(f'$x_{i+1}$(t)')
    axes[i].grid(True, alpha=0.3)
    axes[i].legend()

axes[-1].set_xlabel('Time (s)')
plt.suptitle(f'True Trajectories - {system_name.replace("_", " ").title()}')
plt.tight_layout()
plt.show()

## 2. Model Training

Now let's train a PINN observer on this system with a smaller number of epochs for demonstration.

In [ ]:
# Train the PINN observer
model, best_loss = train(
    system=system_name,
    epochs=10000,  # Reduced for demo
    lr=1e-3,
    Nc=1000,       # Reduced for demo
    Nbc=10,
    t0=0.0,
    t_end=10.0,
    seed=42,
    log_interval=1000,
    weight_ic=1.0,
    weight_pde=1.0,
    weight_y=1.0
)

print(f"Training completed! Best loss: {best_loss:.6e}")

## 3. Model Evaluation

Let's evaluate the trained model and visualize the results.

In [ ]:
# Generate evaluation data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t_eval = np.linspace(0, 10, 500)
t_tensor = torch.tensor(t_eval, dtype=torch.float32, device=device).unsqueeze(1)

# Get predictions
model.eval()
with torch.no_grad():
    out = model(t_tensor)
    x_pred = out[:, :2].cpu().numpy()  # First 2 columns are states
    L_pred = out[:, 2:].view(-1, 2, 1).cpu().numpy()  # Observer gain

print(f"Predicted states shape: {x_pred.shape}")
print(f"Observer gain shape: {L_pred.shape}")

In [ ]:
# Generate true trajectory for comparison
from scipy.integrate import odeint
sys_fn = SYSTEM_MAP[system_name]
x0 = np.array([2, -1])  # Initial condition for modified academic
x_true = odeint(sys_fn, x0, t_eval)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for i in range(2):
    axes[i].plot(t_eval, x_true[:, i], 'r-', linewidth=2, label='True')
    axes[i].plot(t_eval, x_pred[:, i], 'g--', linewidth=2, label='PINN Predicted')
    axes[i].set_ylabel(f'$x_{i+1}$(t)')
    axes[i].grid(True, alpha=0.3)
    axes[i].legend()

axes[-1].set_xlabel('Time (s)')
plt.suptitle('PINN Observer Performance')
plt.tight_layout()
plt.show()

# Calculate and display errors
errors = np.abs(x_true - x_pred)
print(f"Mean absolute errors:")
for i in range(2):
    print(f"  State {i+1}: {np.mean(errors[:, i]):.6f}")

## 4. Observer Gain Analysis

Let's examine how the observer gain matrix evolves over time.

In [ ]:
# Plot observer gain components
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# L is shape (N, 2, 1), so L[:, i, 0] gives the i-th component
for i in range(2):
    axes[i].plot(t_eval, L_pred[:, i, 0], 'b-', linewidth=2)
    axes[i].set_ylabel(f'$L_{i+1}$(t)')
    axes[i].grid(True, alpha=0.3)
    axes[i].set_title(f'Observer Gain Component {i+1}')

axes[-1].set_xlabel('Time (s)')
plt.suptitle('Time-varying Observer Gain Matrix')
plt.tight_layout()
plt.show()

## 5. System Exploration

Let's briefly explore different systems available in the package.

In [ ]:
# Compare trajectories of different systems
systems_to_compare = ["reverse", "academic", "modified_academic"]
fig, axes = plt.subplots(len(systems_to_compare), 2, figsize=(12, 8), sharex=True)

for row, sys_name in enumerate(systems_to_compare):
    # Generate data for this system
    t_comp, x_comp, _, _, _ = generate_dataset(
        name=sys_name, t0=0.0, t_end=5.0, N_colloc=500, N_bc=10, seed=42
    )
    
    # Plot first two states
    for col in range(min(2, x_comp.shape[1])):
        axes[row, col].plot(t_comp, x_comp[:, col], linewidth=2)
        axes[row, col].set_ylabel(f'$x_{col+1}$(t)')
        axes[row, col].grid(True, alpha=0.3)
        if col == 0:
            axes[row, col].text(0.02, 0.95, sys_name.replace('_', ' ').title(), 
                               transform=axes[row, col].transAxes, va='top', 
                               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

axes[-1, 0].set_xlabel('Time (s)')
axes[-1, 1].set_xlabel('Time (s)')
plt.suptitle('Comparison of Different Dynamical Systems')
plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated:

1. **Data Generation**: How to generate training data for different dynamical systems
2. **Model Training**: Training a PINN observer with physics-informed constraints
3. **Evaluation**: Comparing predicted vs. true trajectories
4. **Analysis**: Examining the learned observer gain matrix
5. **System Comparison**: Exploring different available systems

The PINN observer successfully learns to estimate system states while respecting the underlying physics, making it a powerful tool for state estimation in nonlinear dynamical systems.